<a href="https://colab.research.google.com/github/CodeCrafter-101/Audio-to-Text/blob/main/Audio_to_Text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importing the Required Libraries

- `speech_recognition` – for recognizing speech from audio

- `pydub` – for audio processing and splitting on silence

- `google.colab` – for uploading and handling files in Colab

- `subprocess`, `os` – for audio format conversion (e.g., using `ffmpeg`)

In [1]:
pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 14.0 MB/s eta 0:00:00


In [2]:
import speech_recognition as sr
import os
import subprocess
from google.colab import files
from pydub import AudioSegment
from pydub.silence import split_on_silence

Upload Audio File

In [3]:
files.upload()

Saving readpoem.mp3 to readpoem.mp3


{'readpoem.mp3': b'\xff\xf3\x84\xc4\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xf3\x84\xc4\x00&\x83\xe2\x08\x00z\x0c\xdd"\x08X\xec+K\x9bd\xadK\x91\xc0\xe3\x1d\xfc{\xb5:\x7fecZ\xad\x0b\x1e(4\x05\x01a\x88\x93d\n\x07\xf7"\xe7\xe5\xbf\t0)V\x94X\xbb$\x96|\n\x0b\x9e\xe2\xef"\xf6\x96\x94\x89\'

Convert Mp3 to Wav format

In [8]:
# convert mp3 to wav file
subprocess.call(['ffmpeg', '-i', 'readpoem.mp3','wav_file.wav'])

1

Main Function

In [33]:
# Create a recognizer object
r = sr.Recognizer()

# Function to convert large audio file to text
def get_large_audio_transcription(path):
  """
  This function breaks a long audio file into smaller parts
  and converts each part into text using speech recognition.
  """
  # Load the audio file
  sound = AudioSegment.from_wav(path)
  # Split the audio into chunks where there's a pause (silence)
  chunks = split_on_silence(sound,
                            min_silence_len = 700,             # Split if silence is at least 0.5 second
                            silence_thresh = sound.dBFS-10,    # Consider silence if sound is quieter than this
                            keep_silence=700,)                 # Keep a bit of silence at the start and end of each chunk

  folder_name = "audio-chunks"

  # create a directory to store the small audio chunks
  if not os.path.isdir(folder_name):
    os.mkdir(folder_name)

  whole_text = ""

  # Go through each chunk one by one
  for i, audio_chunk in enumerate(chunks, start=1):
      # Save the chunk as a new .wav file
      chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
      audio_chunk.export(chunk_filename, format="wav")

      # Open the chunk to recognize the speech
      with sr.AudioFile(chunk_filename) as source:
          audio_listened = r.record(source)

          # try converting it to text
          try:                                          # Convert audio to text using Google's API
              text = r.recognize_google(audio_listened)
          except sr.UnknownValueError as e:             # If speech wasn't clear, print the error
              # print("Error:", str(e))
              print(f"Skipping chunk{i} - could not understand audio.")
          else:
              text = f"{text.capitalize()}. "           # Capitalize and format the text
              print(chunk_filename, ":", text)
              # Add to the final result
              whole_text += text
  # return the text for all chunks detected
  return whole_text

In [34]:
path = "/content/wav_file.wav"
print("\nFull text:", get_large_audio_transcription(path))

audio-chunks/chunk1.wav : One-sided love people call it one-sided love. 
Skipping chunk2 - could not understand audio.
audio-chunks/chunk3.wav : Heal the cold shoulder that won't. 
audio-chunks/chunk4.wav : Pele we stay close. 
audio-chunks/chunk5.wav : Yet it never reveal i wish i knew how to express. 
audio-chunks/chunk6.wav : But my heart holds still. 
audio-chunks/chunk7.wav : It's just my one-sided love. 
audio-chunks/chunk8.wav : Other yet may never feel a love i feel but it stays unraveled. 

Full text: One-sided love people call it one-sided love. Heal the cold shoulder that won't. Pele we stay close. Yet it never reveal i wish i knew how to express. But my heart holds still. It's just my one-sided love. Other yet may never feel a love i feel but it stays unraveled. 
